In [1]:
from model.LongTermAccompanimentBeatwiseUpcomingBars import LongTermAccompanimentBeatwiseUpcomingBars
from model import load_model
import torch
from hvo_sequence.hvo_seq import HVO_Sequence
from hvo_sequence.drum_mappings import ROLAND_REDUCED_MAPPING
from bokeh.plotting import show, output_notebook
import os
import timeit
from hvo_sequence.hvo_seq import HVO_Sequence

output_notebook()

# mdl_ = 'misc/LTA/(Smaller) Predict 1 bar ahead, no velocity at inputs_w2ujjpkp/100.pth'
# mdl_ = 'misc/LTA/(Smaller) Predict 2 bar ahead, no velocity at inputs_ycwpquxg/100.pth'

mdl_ = 'misc/LTA/(Smaller) Predict 1 bar ahead, no velocity at inputs_5yxlci4h/100.pth'

model = load_model(
    model_path=mdl_,
    model_class=LongTermAccompanimentBeatwiseUpcomingBars,
    device='cpu',
    is_evaluating=True
)

model.serialize(save_folder=os.path.dirname(mdl_), filename=mdl_.split('/')[-1].replace('.pth', '.pt'))
model.eval()

Loading BokehJS ...

max_look_back_segments: 4
n_segments_per_bar:  4


/home/dtic/miniconda3/envs/GrooveTransformer/lib/python3.9/site-packages/torch/jit/_recursive.py:313: UserWarning: 'norm' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(


LongTermAccompanimentBeatwiseUpcomingBars(
  (SegmentEncoder): SegmentEncoder(
    (FCN): Linear(in_features=12, out_features=128, bias=True)
  )
  (PerformanceEncoder): PerformanceEncoder(
    (PositionalEncoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (Encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-3): 4 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False

In [ ]:
# generate a random pattern
hvo = model.generate_random_pattern(n_bars=4)
print(hvo.shape)
# synthesize and display the pattern
hvo_seq = HVO_Sequence(
    beat_division_factors=[4],
    drum_mapping=ROLAND_REDUCED_MAPPING
)
hvo_seq.add_tempo(0, 120)
hvo_seq.add_time_signature(0, 4, 4)
hvo_seq.hvo = hvo[0, :, :].detach().numpy()


# load audio player
from IPython.display import Audio, display
display(Audio(hvo_seq.synthesize(sf_path='hvo_sequence/soundfonts/Standard_Drum_Kit.sf2'), rate=44100))

hvo_seq.to_html_plot(show_figure=True, width=800, height=400)

model.decoder_input_has_velocity



In [ ]:

start = timeit.default_timer()
model.prime_with_drums(hvo)
print('Time: ', (timeit.default_timer() - start) * 1000, 'ms')

In [ ]:
model.generations[:, :16, 3], model.shifted_tgt[:, :17, 3], hvo[0, :16, 3]

In [ ]:
start = timeit.default_timer()
model.encode_input_performance(torch.rand(1, 64, 3))
print('Time: ', (timeit.default_timer() - start) * 1000, 'ms')

In [ ]:
# load data

from data import PairedLTADatasetV2
max_n_bars = 32

test_dataset = PairedLTADatasetV2(
        input_inst_dataset_bz2_filepath="data/lmd/data_bass_groove_test.bz2",
        output_inst_dataset_bz2_filepath="data/lmd/data_drums_full_unsplit.bz2",
        shift_tgt_by_n_steps=1,
        max_input_bars=max_n_bars,
        hop_n_bars=8
    )

In [ ]:
test_dataset_longer = PairedLTADatasetV2(
        input_inst_dataset_bz2_filepath="data/lmd/data_bass_groove_test.bz2",
        output_inst_dataset_bz2_filepath="data/lmd/data_drums_full_unsplit.bz2",
        shift_tgt_by_n_steps=1,
        max_input_bars=128,
        hop_n_bars=32
    )

In [ ]:
from model.LongTermAccompanimentBeatwiseUpcomingBars import LongTermAccompanimentBeatwiseUpcomingBars
from model import load_model
import torch
from hvo_sequence.hvo_seq import HVO_Sequence
from hvo_sequence.drum_mappings import ROLAND_REDUCED_MAPPING
from bokeh.plotting import show, output_notebook
import os
import timeit
from hvo_sequence.hvo_seq import HVO_Sequence

len_segs = 60
model.reset_all()
sample_ix = torch.randint(0, len(test_dataset_longer), (1,)).item()
input_ = test_dataset_longer.instrument1_hvos[sample_ix].unsqueeze(0)[:, :len_segs*4, :]
# model.prime_with_drums(test_dataset_longer.instrument2_hvos[sample_ix].unsqueeze(0)[:, :32, :])
gens = torch.zeros((1, input_.shape[1]+32, 27))
print(gens.shape)

start = timeit.default_timer()
for i in range(0, len_segs):#input_.shape[1] // 4 ):
    if (i % 4) == 0:
        print(f"-------------------Bar {i // 4}-------------------")
    next_K_bars = model.predict_next_K_bars_starting_at(start_=i * 4, threshold=0.05, print_info=True)
    gens[:, i*4:(i*4+next_K_bars.shape[1]), :] = next_K_bars
    model.encode_input_performance(input_[:, i*4:(i+1)*4, :])
        
print('Time: ', (timeit.default_timer() - start) * 1000, 'ms')


# play and display the generated pattern
hvo_seq_bass = HVO_Sequence(
    beat_division_factors=[4],
    drum_mapping={'BASS Rhythm': [80]}
)
hvo_seq_bass.add_tempo(0, 120)
hvo_seq_bass.add_time_signature(0, 4, 4)
hvo_seq_bass.hvo = input_[0, :, :].detach().numpy()

hvo_seq = HVO_Sequence(
    beat_division_factors=[4],
    drum_mapping=ROLAND_REDUCED_MAPPING
    
    
)
hvo_seq.add_tempo(0, 120)
hvo_seq.add_time_signature(0, 4, 4)
print(input_.shape[1], 48 * 4)
hvo_seq.hvo = gens[0, :input_.shape[1], :].detach().numpy()

audio1 = hvo_seq.synthesize(sf_path='hvo_sequence/soundfonts/Standard_Drum_Kit.sf2')
audio2 = hvo_seq_bass.synthesize(sf_path='hvo_sequence/soundfonts/Standard_Drum_Kit.sf2',track_n=9, program=0)
audio1 = audio1[:min(audio1.shape[0], audio2.shape[0])]
audio2 = audio2[:min(audio1.shape[0], audio2.shape[0])] * 0.6
display(Audio(audio1+audio2, rate=44100))

hvo_seq_bass.to_html_plot(show_figure=True, width=1400, height=150, filename='Original Bass')
hvo_seq.to_html_plot(show_figure=True, width=1400, height=400)

In [ ]:
def batch_data_extractor(data_, device='cpu'):

    bass_solo = data_[0].to(device) if data_[0].device.type != device else data_[0]
    drums = data_[1].to(device) if data_[1].device.type != device else data_[1]
    stacked_bass_drums = data_[2].to(device) if data_[2].device.type != device else data_[2]
    shifted_drums = data_[3].to(device) if data_[3].device.type != device else data_[3]

    return bass_solo, drums, stacked_bass_drums, shifted_drums


def predict_using_batch_data(batch_data, model_=model, device='cpu', mute_bass=False, max_n_bars=32):
    model_.eval()



    bass_solo, drums, stacked_bass_drums, shifted_drums = batch_data_extractor(
        data_=batch_data,
        device=device
    )



    with torch.no_grad():
        h, v, o, hvo, h_logits = model_.sample(
            src=bass_solo[:, :max_n_bars*16, :] if not mute_bass else torch.zeros_like(bass_solo[:, :max_n_bars*16, :]),
            tgt=shifted_drums[:, :max_n_bars*16, :],

        )
    return hvo, h_logits

In [ ]:
sample_ix = torch.randint(0, len(test_dataset), (1,)).item()
bass_solo, drums, stacked_bass_drums, shifted_drums = batch_data_extractor(
    data_=test_dataset[sample_ix],
    device='cpu'
)
shifted_drums[1, :] == drums[0, :]


In [ ]:


# select a random sample
import numpy as np
import timeit
sample_ix = np.random.randint(0, len(test_dataset))
print(sample_ix)

hvo, h_logits = predict_using_batch_data(test_dataset[sample_ix:sample_ix+1], model_=model, max_n_bars=32, mute_bass=False)

hvo_gt = test_dataset.instrument2_hvos[sample_ix]
hvo_shifted = test_dataset.instrument2_shifted_hvos[sample_ix]


total = torch.cat([hvo[0], hvo_gt[:32, :]], dim=0)


hvo_seq = HVO_Sequence(
    beat_division_factors=[4],
    drum_mapping=ROLAND_REDUCED_MAPPING,
)
hvo_seq.add_tempo(0, 120)
hvo_seq.add_time_signature(0, 4, 4) 
print(total.cpu().numpy().shape) 
hvo_seq.hvo = total.cpu().numpy()
hvo_seq.to_html_plot(show_figure=True, width=1400, height=400)

hvo[0, 0, :]


In [ ]:
hvo_seq2 = HVO_Sequence(
    beat_division_factors=[4], 
    drum_mapping=ROLAND_REDUCED_MAPPING
)
hvo_seq2.add_tempo(0, 120)
hvo_seq2.add_time_signature(0, 4, 4)
hvo_seq2.hvo = torch.cat([hvo[0]], dim=0).cpu().numpy()

# load audio player
from IPython.display import Audio
Audio(hvo_seq2.synthesize(sf_path='hvo_sequence/soundfonts/Standard_Drum_Kit.sf2'), rate=44100)


In [ ]:


# hvo_seq = HVO_Sequence(
#     beat_division_factors=[4],
#     drum_mapping=ROLAND_REDUCED_MAPPING
# )
# hvo_seq.add_tempo(0, 120)
# hvo_seq.add_time_signature(0, 4, 4) 
# print(total.cpu().numpy().shape) 
# hvo_seq.hvo = hvo_shifted.cpu().numpy()
# hvo_seq.to_html_plot(show_figure=True, width=1400, height=400)
# 
# hvo_seq = HVO_Sequence(
#     beat_division_factors=[4],
#     drum_mapping=ROLAND_REDUCED_MAPPING
# )
# hvo_seq.add_tempo(0, 120)
# hvo_seq.add_time_signature(0, 4, 4) 
# print(total.cpu().numpy().shape) 
# hvo_seq.hvo = hvo_gt.cpu().numpy()
# hvo_seq.to_html_plot(show_figure=True, width=1400, height=400)



In [ ]:

# Auto-regressive prediction
def predict_using_batch_data_auto_Reg(batch_data, model_=model, device='cpu', max_n_bars=32, mute_bass=False,
                                      prime_with_this_hvo=None, mask_bass_vel=True, mask_dec_in_vel=True, use_bernulli=True, temperature=1.0, threshold=0.5, reproduceable=False, 
                                      remove_kicks=False, remove_snares=False, remove_hats=False, remove_toms=False, remove_crashs=False, remove_rides=False):
    model_.eval()
    
    assert not (reproduceable and use_bernulli), "Cannot use be reproduceable if use_bernulli"
    
    bass_solo_, drums_, stacked_bass_drums_, shifted_drums_ = batch_data_extractor(
        data_=batch_data,
        device=device
    )

    bass_solo = bass_solo_.clone()
    shifted_drums = shifted_drums_.clone()
    
    if mask_dec_in_vel:
        shifted_drums[:, :, 9:18] = 0
         
    if isinstance(mask_bass_vel, bool) and mask_bass_vel:
        bass_solo[:, :, 1] = 0
    elif isinstance(mask_bass_vel, float):
        bass_solo[:, :, 1] *= mask_bass_vel

    
    if prime_with_this_hvo is None:
        shifted_in = torch.zeros((1, (max_n_bars) * 16 + 1, 27))
        gens = torch.zeros((1, (max_n_bars) * 16, 27))
        start_at = 0
    else:
        assert prime_with_this_hvo.shape[0] % 16 == 0
        shifted_in = torch.zeros((1, max_n_bars * 16 + prime_with_this_hvo.shape[0] + 1, 27))
        gens = torch.zeros((1, max_n_bars * 16  + prime_with_this_hvo.shape[0], 27))
        start_at = prime_with_this_hvo.shape[0]
        shifted_in[:, 1:start_at+1, :] = prime_with_this_hvo.unsqueeze(0)[:, :, :]
        gens[:, :start_at, :] = prime_with_this_hvo.unsqueeze(0)[:, :, :]
        

    start = timeit.default_timer()
    print("Starting prediction at bar: ", start_at)
    for i in range(start_at, max_n_bars * 16):
        n_quarter_notes = int(i // 4) + 1
        with torch.no_grad():
            if (i - start_at) < 32 and not reproduceable:
                thresh = 0.05
                sample_bernulli = use_bernulli
            else:
                thresh = threshold
                sample_bernulli = use_bernulli

            h, v, o, hvo_, h_logits = model_.sample(
                src=bass_solo[:, :n_quarter_notes * 4, :] if not mute_bass else torch.zeros_like(bass_solo[:, :n_quarter_notes * 4, :]),
                tgt=shifted_in[:, :i+1, :],
                scale_vel=1.0,
                threshold=thresh,
                use_bernulli=sample_bernulli,
                temperature=temperature
            )
            
            if remove_kicks:
                hvo_[:, :, 0::9] = 0
            if remove_snares:
                hvo_[:, :, 1::9] = 0
            if remove_hats:
                hvo_[:, :, 2::9] = 0
                hvo_[0, :, 3::9] = 0
            if remove_toms:
                hvo_[:, :, 4::9] = 0
                hvo_[:, :, 5::9] = 0
                hvo_[:, :, 6::9] = 0
            if remove_crashs:
                hvo_[:, :, 7::9] = 0
            if remove_rides:
                hvo_[:, :, 8::9] = 0
        
        gens[:, i, :] = hvo_[:, i, :].clone()
        shifted_in[:, i+1, :] = hvo_[:, i, :]
        shifted_in[:, :,  9:18] = 0

    print('Time: ', (timeit.default_timer() - start) * 1000, 'ms')
        
    return gens, h_logits

In [ ]:
import numpy as np
import timeit

# sample_ix = np.random.randint(0, len(test_dataset))
# select a random 2 bar sample 
sample_ix = np.random.randint(0, len(test_dataset)) #2373, 3819,1220, 3782, 1539, 1966, 2902, 2507
print(sample_ix)
random_hvo = test_dataset.instrument2_hvos[np.random.randint(0, len(test_dataset))][-32:, :]
primer_hvo = test_dataset.instrument2_hvos[sample_ix][:32, :]

use_bernulli = True
reproduceable = False
temperature = 1
threshold = 0.0

hvo, h_logits_w_bass = predict_using_batch_data_auto_Reg(
    test_dataset[sample_ix:sample_ix+1], model_=model, max_n_bars=32, mute_bass=False, prime_with_this_hvo=None,
    mask_bass_vel=True, mask_dec_in_vel=True,
    use_bernulli=use_bernulli, temperature=temperature, threshold=threshold, reproduceable=reproduceable, remove_kicks=False, remove_snares=False, remove_hats=False, remove_toms=False, remove_crashs=True, remove_rides=False)

# hvo_gt = test_dataset.instrument2_hvos[sample_ix][-64:, :]



total = torch.cat([hvo[0, :, :]], dim=0)


hvo_seq = HVO_Sequence(
    beat_division_factors=[4],
    drum_mapping=ROLAND_REDUCED_MAPPING
)

hvo_seq.add_tempo(0, 120)
hvo_seq.add_time_signature(0, 4, 4)
print(total.cpu().numpy().shape)
hvo_seq.hvo = total.cpu().numpy()

bass_map = {'BASS Rhythm': [80]}

hvo_seq_bass = HVO_Sequence(
    beat_division_factors=[4],
    drum_mapping=bass_map
)
hvo_seq_bass.add_tempo(0, 120)
hvo_seq_bass.add_time_signature(0, 4, 4)
hvo_seq_bass.hvo = test_dataset.instrument1_hvos[sample_ix].cpu().numpy()

# load audio player
from IPython.display import Audio, display
audio1 = hvo_seq.synthesize(sf_path='hvo_sequence/soundfonts/Standard_Drum_Kit.sf2')
audio2 = hvo_seq_bass.synthesize(sf_path='hvo_sequence/soundfonts/Standard_Drum_Kit.sf2',track_n=9, program=0)
audio1 = audio1[:min(audio1.shape[0], audio2.shape[0])]
audio2 = audio2[:min(audio1.shape[0], audio2.shape[0])] * 0.6
display(Audio(audio1+audio2, rate=44100))

hvo_seq_bass.to_html_plot(show_figure=True, width=1400, height=150, filename='Original Bass')

hvo_seq.to_html_plot(show_figure=True, width=1400, height=400, filename='Drums With Original Bass')

hvo[0, 0, :]

In [ ]:
# show logits above one another as a heatmap
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd



h_probs_w_bass = torch.sigmoid(h_logits_w_bass[0, :, :]).cpu().numpy().transpose()

fig, ax = plt.subplots(1, 1, figsize=(20, 2))
# use a grey scale colormap with white being the lowest value

# h_probs_w_bass[h_probs_w_bass < threshold] = 0

sns.heatmap(h_probs_w_bass, ax=ax, cmap='Greys')

# flip the y-axis so that the first row of the matrix is at the top
ax.invert_yaxis()

plt.show()

In [ ]:
h_probs_w_bass